In [1]:
#### WRITE three sentence synopsis of this technique for Abstract ####

#### also write a two to three paragraph Wiki type thing for future students to grasp what the SVM ML alg does ####


import pandas
from sklearn import linear_model as lm
from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn import preprocessing as pp
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import mean_squared_error
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

import csv
import re

myFile = open("Leptin.txt", "r")

dnaSeq = myFile.read()
#print dnaSeq

myFile.close()

origin = dnaSeq.find( 'ORIGIN' )

startSeq = dnaSeq.find ('1', origin)

endSeq = dnaSeq.find ('//', origin)

extractedSeq = dnaSeq[startSeq:endSeq].split ('\n')

extractedDNA = ""

for eachline in extractedSeq:
    sublist = eachline.split()
    extractedDNA += "".join(sublist[1:])
######## Paige stopped collaborating with us here #########

#find the location numbers where splice sites occur
#got this info from the Leptin.txt PDB file
mRNA = dnaSeq.find('mRNA')
join = dnaSeq.find('join(', mRNA)
joinEnd = dnaSeq.find(')', mRNA)
#print dnaSeq[join+5]

#put these numbers in more easily accessible list
sequence = dnaSeq[join+5:joinEnd].split('..')
print sequence

##### Kayla and I stopped collaborating here #####
myList = [i.split(',') for i in sequence]
print myList

#final version of list with the locations of true splice sites
subList = []
for x in myList:
    #print x
    for y in x:
        #print y
        subList.append(y)
print subList

#initialize the outputted csv file
outfile = open('leptin.csv','w')
fieldnames = ['seq','orf']
csvwriter = csv.DictWriter(outfile, fieldnames=fieldnames)
csvwriter.writeheader()

#print out to csv -- done
#n = number of base pairs on either side
n=30
for x in subList:
    subseq = extractedDNA[int(x)-n:int(x)+n]
    subseq = [w.replace('a', '1000') for w in subseq]
    subseq = [w.replace('c', '0100') for w in subseq]
    subseq = [w.replace('g', '0010') for w in subseq]
    subseq = [w.replace('t', '0001') for w in subseq]
    subseq = ''.join(subseq)
    #outfile.write([(extractedDNA[int(x)-n:int(x)+n]), '1'])
    #csvwriter.writerow({'seq':(extractedDNA[int(x)-n:int(x)+n]), 'orf':'1'})
    csvwriter.writerow({'seq':(subseq), 'orf':'1'})
    
#find location of AG
#create a sublist and add this to the csv file
#30 total on each side, starting after "ag" or as far as you can go -- make up difference on opposite side
#go through extractedDNA, find the position of ag
#then, take 30 on each side of that position 

#this finds one site -- the first site
#notSites = extractedDNA.find('ag')
#print notSites #the position right before ag occurs

#hopefully finds ag
#i used regular expressions and didn't burn myself!! yay!!!!
pattern = re.compile(r'(ag)')

#gives number of times we find ag in extractedDNA
hits = re.finditer(pattern,extractedDNA)

#find the location of beginning of ag for each hit
for match in hits:
    #madHitz is ag
    madHitz = match.group()
    #starting location of ag
    hit_start = match.start()
    #print madHitz, hit_start
    #my math wizardry worked!!!!! 
    if hit_start >= 30 and hit_start <= (len(extractedDNA) - n) and hit_start not in subList:
        subseq1 = extractedDNA[int(hit_start)-n:int(hit_start)+n]
        subseq1 = [w.replace('a', '1000') for w in subseq1]
        subseq1 = [w.replace('c', '0100') for w in subseq1]
        subseq1 = [w.replace('g', '0010') for w in subseq1]
        subseq1 = [w.replace('t', '0001') for w in subseq1]
        subseq1 = ''.join(subseq1)
        #csvwriter.writerow({'seq':(extractedDNA[int(hit_start)-n:int(hit_start)+n]), 'orf':'0'})
        csvwriter.writerow({'seq':(subseq1), 'orf':'0'})
    if hit_start < 30:
        subseq2 = extractedDNA[int(hit_start)-int(hit_start):int(hit_start)+n+(n-int(hit_start))]
        subseq2 = [w.replace('a', '1000') for w in subseq2]
        subseq2 = [w.replace('c', '0100') for w in subseq2]
        subseq2 = [w.replace('g', '0010') for w in subseq2]
        subseq2 = [w.replace('t', '0001') for w in subseq2]
        subseq2 = ''.join(subseq2)
        #csvwriter.writerow({'seq':(extractedDNA[int(hit_start)-int(hit_start):int(hit_start)+n+(n-int(hit_start))]), 'orf':'0'})
        csvwriter.writerow({'seq':(subseq2), 'orf':'0'})
    if hit_start >= (len(extractedDNA) - n):
        subseq3 = extractedDNA[int(hit_start)-(n+n-(len(extractedDNA)-int(hit_start))):int(hit_start)+len(extractedDNA)-int(hit_start)]
        subseq3 = [w.replace('a', '1000') for w in subseq3]
        subseq3 = [w.replace('c', '0100') for w in subseq3]
        subseq3 = [w.replace('g', '0010') for w in subseq3]
        subseq3 = [w.replace('t', '0001') for w in subseq3]
        subseq3 = ''.join(subseq3)
        #csvwriter.writerow({'seq':(extractedDNA[int(hit_start)-(n+n-(len(extractedDNA)-int(hit_start))):int(hit_start)+len(extractedDNA)-int(hit_start)]), 'orf':'0'})
        csvwriter.writerow({'seq':(subseq3), 'orf':'0'})

        
################################## LINEAR SVC ##############################

#I now have the entire list -- both yes and no examples. I only have 6 yes examples though; the rest are no. 
#One yes example equals a no example. Need to root this out.
#genome_data = pandas.read_csv('leptin.csv')
myFile = open("splice_data.csv", "r")

spliceSeq = myFile.read()
#print spliceSeq

spliceSeq = [w.replace('a', '1000') for w in spliceSeq]
spliceSeq = [w.replace('c', '0100') for w in spliceSeq]
spliceSeq = [w.replace('g', '0010') for w in spliceSeq]
spliceSeq = [w.replace('t', '0001') for w in spliceSeq]
spliceSeq = ''.join(spliceSeq)

#print spliceSeq

#outfile = open('splice_data2.csv','w')
#csvwriter.writerow(spliceSeq)
with open('splice_data2.csv','w') as f:
    for item in spliceSeq:
        f.write(item)

genome_data = pandas.read_csv('splice_data2.csv')

avg = 0
for i in range(0,100):
    # now to test the thing -- most will be no, though. how do i split it w/ only three examples?
    (genome_train,genome_test) = cv.train_test_split(genome_data,test_size=0.2)

    #perc_alg = svm.PolynomialSVC()

    perc_alg = MultinomialNB() 
    
    predictors = ["seq"]  
    perc_alg.fit(genome_train[predictors],genome_train["orf"])

    #predicts the test data 
    genome_test_preds = perc_alg.predict(genome_test[predictors])


    #print metrics.accuracy_score(genome_test["orf"],genome_test_preds)
    avg = float(avg) + float(metrics.accuracy_score(genome_test["orf"],genome_test_preds))
print float(avg/100)

print metrics.classification_report(genome_test["orf"],genome_test_preds)

############################################################################




['5001', '5029,15714', '15885,18127', '21352']
[['5001'], ['5029', '15714'], ['15885', '18127'], ['21352']]
['5001', '5029', '15714', '15885', '18127', '21352']


AttributeError: 'module' object has no attribute 'PolynomialSVC'